In [1]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

from model import *
from utils import *

In [2]:
blocks = parse_cfg('./cfg/model.cfg')
net_info, module_list = create_modules(blocks)

In [9]:
module = module_list[0]
module

Sequential(
  (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_0): LeakyReLU(negative_slope=0.1, inplace=True)
)

## Weights of first conv block before loading

In [10]:
module[0].weight[0]

tensor([[[ 0.1242,  0.1383, -0.1402],
         [ 0.1687, -0.1114, -0.1365],
         [-0.1868, -0.0517, -0.1537]],

        [[ 0.0248, -0.0960, -0.0430],
         [-0.1645,  0.0467,  0.0498],
         [ 0.0023, -0.1888,  0.0903]],

        [[ 0.0002, -0.0594,  0.0481],
         [-0.1507, -0.1015,  0.0061],
         [-0.0938, -0.0025, -0.1516]]], grad_fn=<SelectBackward0>)

## Weights of first conv block after loading

In [4]:
model = Net("cfg/model.cfg")
model.load_weights("weights/yolov3.weights")

In [12]:
model.module_list[0][0].weight[0]

tensor([[[-0.0740,  0.1793, -0.0300],
         [ 0.0912, -0.0169,  0.0980],
         [ 0.0027, -0.7275,  0.0275]],

        [[-0.0436, -0.1081, -0.0277],
         [ 0.0375, -0.1492,  0.0688],
         [ 0.0402, -0.4120,  0.0622]],

        [[ 0.0263,  0.1104, -0.0717],
         [ 0.0608,  0.0507,  0.0366],
         [-0.0071, -0.4448,  0.0145]]], grad_fn=<SelectBackward0>)

## Test Foward Pass

In [44]:
net = Net("cfg/model.cfg")
input = test_forward_pass()
pred = net(input, torch.cuda.is_available())
pred

tensor([[[2.2937e+01, 1.5843e+01, 9.5361e+01,  ..., 4.2360e-01,
          4.2999e-01, 4.6617e-01],
         [2.0630e+01, 2.0300e+01, 3.1507e+02,  ..., 4.3634e-01,
          4.5117e-01, 5.2884e-01],
         [2.2486e+01, 2.4395e+01, 3.7954e+02,  ..., 5.4638e-01,
          5.4957e-01, 5.7353e-01],
         ...,
         [5.6665e+02, 5.6660e+02, 8.9677e+00,  ..., 4.8677e-01,
          3.8770e-01, 4.3729e-01],
         [5.6562e+02, 5.6691e+02, 2.2953e+01,  ..., 5.1666e-01,
          5.3433e-01, 5.6511e-01],
         [5.6569e+02, 5.6543e+02, 4.0473e+01,  ..., 5.2003e-01,
          4.6440e-01, 6.1079e-01]]])

In [43]:
pred.shape

torch.Size([1, 10647, 85])

size: batch_size x n_bboxes x (4 bbox attrs + 1 object score + 80 class scores)